# API for Game Data

In [1]:
import requests
import pandas as pd

def get_nfl_games(year: int) -> pd.DataFrame:

    header = {"User-Agent": "xcs5hg@virginia.edu"}

    def get_json(url: str):
        r = requests.get(url, headers=header)
        r.raise_for_status()
        return r.json()

    season_url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{year}"
    season_data = get_json(season_url)

    season_parts = {}  

    for t in season_data["types"]["items"]:
        t_data = get_json(t["$ref"])
        name = t_data.get("name")
        if name in ["Regular Season", "Postseason"]:
            season_parts[name] = t_data

    if not season_parts:
        raise ValueError(f"No Regular/Postseason types found for {year}")

    event_urls = []

    for part_name, part in season_parts.items():
        weeks_url = part["weeks"]["$ref"]
        weeks_data = get_json(weeks_url)
        week_items = weeks_data.get("items", [])

        for w in week_items:
            week = get_json(w["$ref"])
            events_url = week["events"]["$ref"]
            events_data = get_json(events_url)
            event_urls.extend(e["$ref"] for e in events_data.get("items", []))

    records = []

    for event_url in event_urls:
        event = get_json(event_url)

        game_id = event.get("id")
        date = event.get("date")

        comp = event["competitions"][0]
        competitors = comp.get("competitors", [])

        home_team = None
        away_team = None

        for c in competitors:
            side = c.get("homeAway") 
            team_obj = c.get("team", {})

            if "$ref" in team_obj:
                team_data = get_json(team_obj["$ref"])
            else:
                team_data = team_obj

            team_name = (
                team_data.get("displayName")
                or team_data.get("name")
                or team_data.get("abbreviation")
            )

            if side == "home":
                home_team = team_name
            elif side == "away":
                away_team = team_name

        records.append(
            {
                "game_id": game_id,
                "date": date,
                "home_team": home_team,
                "away_team": away_team,
            }
        )

    return pd.DataFrame(records)

In [30]:
df_games_2019

NameError: name 'df_games_2024' is not defined

In [31]:
# API for play by play AND API for each season's team's rooster

NameError: name 'build_nfl_games_df' is not defined

In [2]:
import requests
import pandas as pd

def get_game_plays(game_id: int) -> pd.DataFrame:

    header = {"User-Agent": "xcs5hg@virginia.edu"}

    event_url = (
        "https://sports.core.api.espn.com/v2/"
        f"sports/football/leagues/nfl/events/{game_id}"
    )
    event = requests.get(event_url, headers=header).json()

    competition = event["competitions"][0]
    comp_id = competition["id"]

    plays_url = (
        "https://sports.core.api.espn.com/v2/"
        f"sports/football/leagues/nfl/events/{game_id}/competitions/{comp_id}/plays?limit=500"
    )

    plays = requests.get(plays_url, headers=header).json()

    record = [item for item in plays["items"]]

    keep_cols = [
        "text",
        "period.number",
        "clock.value",
        "start.down",
        "start.distance",
        "start.yardLine",
        "start.yardsToEndzone",
        "end.yardsToEndzone",
        "homeScore",
        "awayScore",
        "statYardage",
        "scoringPlay",
        "scoreValue",
        "type.text",
        "type.abbreviation",
        "team.$ref",       # NEW
        "participants",    # NEW (list of dicts)
    ]

    data = pd.json_normalize(record)

    # only keep the columns that actually exist (to be safe)
    cols_existing = [c for c in keep_cols if c in data.columns]
    data = data[cols_existing]

    return data

In [32]:
import math

header = {"User-Agent": "xcs5hg@virginia.edu"}

def get_json(url: str):
    r = requests.get(url, headers=header)
    r.raise_for_status()
    return r.json()

def get_game_roster_from_plays(game_id: int) -> pd.DataFrame:
    """
    Use get_game_plays(game_id) (above) to infer the set of players
    who appeared in that game.
    """

    plays_df = get_game_plays(game_id)
    if plays_df.empty:
        return pd.DataFrame()

    # 1) Build mapping team_ref -> (team_id, team_name)
    team_refs = (
        plays_df.get("team.$ref")
        .dropna()
        .unique()
        .tolist()
        if "team.$ref" in plays_df.columns
        else []
    )

    team_map = {}
    for tref in team_refs:
        if isinstance(tref, str):
            tdata = get_json(tref)
            tid = tdata.get("id")
            tname = (
                tdata.get("displayName")
                or tdata.get("name")
                or tdata.get("abbreviation")
            )
            team_map[tref] = (tid, tname)

    athlete_cache = {}
    rows = []

    for _, row in plays_df.iterrows():
        team_ref = row.get("team.$ref")
        team_id, team_name = (None, None)
        if isinstance(team_ref, str) and team_ref in team_map:
            team_id, team_name = team_map[team_ref]

        participants = row.get("participants")

        # handle NaN / None
        if participants is None or (isinstance(participants, float) and math.isnan(participants)):
            continue
        if not isinstance(participants, list):
            continue

        for part in participants:
            if not isinstance(part, dict):
                continue

            athlete_ref = part.get("athlete", {}).get("$ref")
            if not athlete_ref:
                continue

            if athlete_ref in athlete_cache:
                athlete = athlete_cache[athlete_ref]
            else:
                athlete = get_json(athlete_ref)
                athlete_cache[athlete_ref] = athlete

            pos = athlete.get("position") or {}

            rows.append(
                {
                    "game_id": game_id,
                    "team_id": team_id,
                    "team_name": team_name,
                    "athlete_id": athlete.get("id"),
                    "full_name": athlete.get("fullName"),
                    "display_name": athlete.get("displayName"),
                    "position_name": pos.get("name"),
                    "position_abbr": pos.get("abbreviation"),
                }
            )

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    # 3) Deduplicate per game/team/athlete
    df = df.drop_duplicates(subset=["game_id", "team_id", "athlete_id"])
    return df

In [33]:
plays_1 = get_game_plays(401127913)

roster_1 = get_game_roster_from_plays(401127913)


(89,
      game_id team_id          team_name athlete_id             full_name  \
 0  401127913       9  Green Bay Packers    4034949          Eddy Pineiro   
 1  401127913       9  Green Bay Packers    3042519       Aaron Jones Sr.   
 2  401127913       9  Green Bay Packers    3915189          Roquan Smith   
 3  401127913       9  Green Bay Packers       8439         Aaron Rodgers   
 7  401127913       9  Green Bay Packers    2574891  Roy Robertson-Harris   
 
            display_name     position_name position_abbr  
 0          Eddy Pineiro      Place kicker            PK  
 1       Aaron Jones Sr.      Running Back            RB  
 2          Roquan Smith        Linebacker            LB  
 3         Aaron Rodgers       Quarterback            QB  
 7  Roy Robertson-Harris  Defensive Tackle            DT  )

In [34]:
pd.json_normalize(odds['odds']['items'])

,$ref,details,overUnder,spread,overOdds,underOdds,links,moneylineWinner,spreadWinner,provider.$ref,provider.id,provider.name,provider.priority,awayTeamOdds.favorite,awayTeamOdds.underdog,awayTeamOdds.moneyLine,awayTeamOdds.spreadOdds,awayTeamOdds.open.favorite,awayTeamOdds.open.pointSpread.alternateDisplayValue,awayTeamOdds.open.pointSpread.american,awayTeamOdds.open.spread.value,awayTeamOdds.open.spread.displayValue,awayTeamOdds.open.spread.alternateDisplayValue,awayTeamOdds.open.spread.decimal,awayTeamOdds.open.spread.fraction,awayTeamOdds.open.spread.american,awayTeamOdds.open.moneyLine.value,awayTeamOdds.open.moneyLine.displayValue,awayTeamOdds.open.moneyLine.alternateDisplayValue,awayTeamOdds.open.moneyLine.decimal,awayTeamOdds.open.moneyLine.fraction,awayTeamOdds.open.moneyLine.american,awayTeamOdds.close.pointSpread.alternateDisplayValue,awayTeamOdds.close.pointSpread.american,awayTeamOdds.close.spread.value,awayTeamOdds.close.spread.displayValue,awayTeamOdds.close.spread.alternateDisplayValue,awayTeamOdds.close.spread.decimal,awayTeamOdds.close.spread.fraction,awayTeamOdds.close.spread.american,awayTeamOdds.close.moneyLine.value,awayTeamOdds.close.moneyLine.displayValue,awayTeamOdds.close.moneyLine.alternateDisplayValue,awayTeamOdds.close.moneyLine.decimal,awayTeamOdds.close.moneyLine.fraction,awayTeamOdds.close.moneyLine.american,awayTeamOdds.current.pointSpread.alternateDisplayValue,awayTeamOdds.current.pointSpread.american,awayTeamOdds.current.spread.value,awayTeamOdds.current.spread.displayValue,awayTeamOdds.current.spread.alternateDisplayValue,awayTeamOdds.current.spread.decimal,awayTeamOdds.current.spread.fraction,awayTeamOdds.current.spread.american,awayTeamOdds.current.spread.outcome.type,awayTeamOdds.current.moneyLine.value,awayTeamOdds.current.moneyLine.displayValue,awayTeamOdds.current.moneyLine.alternateDisplayValue,awayTeamOdds.current.moneyLine.decimal,awayTeamOdds.current.moneyLine.fraction,awayTeamOdds.current.moneyLine.american,awayTeamOdds.current.moneyLine.outcome.type,awayTeamOdds.team.$ref,homeTeamOdds.favorite,homeTeamOdds.underdog,homeTeamOdds.moneyLine,homeTeamOdds.spreadOdds,homeTeamOdds.open.favorite,homeTeamOdds.open.pointSpread.value,homeTeamOdds.open.pointSpread.displayValue,homeTeamOdds.open.pointSpread.alternateDisplayValue,homeTeamOdds.open.pointSpread.decimal,homeTeamOdds.open.pointSpread.fraction,homeTeamOdds.open.pointSpread.american,homeTeamOdds.open.spread.value,homeTeamOdds.open.spread.displayValue,homeTeamOdds.open.spread.alternateDisplayValue,homeTeamOdds.open.spread.decimal,homeTeamOdds.open.spread.fraction,homeTeamOdds.open.spread.american,homeTeamOdds.open.moneyLine.value,homeTeamOdds.open.moneyLine.displayValue,homeTeamOdds.open.moneyLine.alternateDisplayValue,homeTeamOdds.open.moneyLine.decimal,homeTeamOdds.open.moneyLine.fraction,homeTeamOdds.open.moneyLine.american,homeTeamOdds.close.pointSpread.alternateDisplayValue,homeTeamOdds.close.pointSpread.american,homeTeamOdds.close.spread.value,homeTeamOdds.close.spread.displayValue,homeTeamOdds.close.spread.alternateDisplayValue,homeTeamOdds.close.spread.decimal,homeTeamOdds.close.spread.fraction,homeTeamOdds.close.spread.american,homeTeamOdds.close.moneyLine.value,homeTeamOdds.close.moneyLine.displayValue,homeTeamOdds.close.moneyLine.alternateDisplayValue,homeTeamOdds.close.moneyLine.decimal,homeTeamOdds.close.moneyLine.fraction,homeTeamOdds.close.moneyLine.american,homeTeamOdds.current.pointSpread.alternateDisplayValue,homeTeamOdds.current.pointSpread.american,homeTeamOdds.current.spread.value,homeTeamOdds.current.spread.displayValue,homeTeamOdds.current.spread.alternateDisplayValue,homeTeamOdds.current.spread.decimal,homeTeamOdds.current.spread.fraction,homeTeamOdds.current.spread.american,homeTeamOdds.current.spread.outcome.type,homeTeamOdds.current.moneyLine.value,homeTeamOdds.current.moneyLine.displayValue,homeTeamOdds.current.moneyLine.alternateDisplayValue,homeTeamOdds.current.moneyLine.decimal,homeTeamOdds.current.money

In [ ]:
##  Lou  ##
#Testing Head to Head odds
sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{EVENT_ID}/competitions/{EVENT_ID}/odds/{BET_PROVIDER_ID}/head-to-heads



In [ ]:
import requests

header = {"User-Agent": "xcs5hg@virginia.edu"}

def get_scoreboard(start_date: str, end_date: str | None = None, limit: int = 1000):

    base_url = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard"
    header = {"User-Agent": "xcs5hg@virginia.edu"}


    # Build dates parameter: either 'YYYYMMDD' or 'YYYYMMDD-YYYYMMDD'
    if end_date is not None:
        dates_param = f"{start_date}-{end_date}"
    else:
        dates_param = start_date  # can be 'YYYY', 'YYYYMMDD', etc.

    params = {
        "limit": limit,
        "dates": dates_param,
        # optional cache-buster if you want:
        # "_": "1577413600",
    }

    r = requests.get(base_url, headers=header, params=params)
    r.raise_for_status()
    return r.json()

# Athletes API

In [56]:
import requests

header = {"User-Agent": "xcs5hg@virginia.edu"}

def get_active_athletes(max_athletes: int = 10000):
    """
    Return a list of $ref URLs for all active NFL athletes (up to max_athletes),
    following ESPN's pagination.
    """
    url = "https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes"
    params = {
        "active": "true",
        "limit": 20000,  
    }
    header = {"User-Agent": "xcs5hg@virginia.edu"}

    date = r.get("meta", {}).get("firstPlayWallClock", "")[:10]

    while url and len(records) < max_athletes:
        resp = requests.get(url, headers=header, params=params)
        resp.raise_for_status()
        data = resp.json()

        for item in data.get("items", []):
            records.append(item["$ref"])
            if len(records) >= max_athletes:
                break

        next_ref = data.get("next", {}).get("$ref")
        if next_ref and len(records) < max_athletes:
            url = next_ref
            params = None 
        else:
            url = None

    players = pd.DataFrame()
    print(len(players))
    
    for i in range(0,50):
        player = requests.get(records[i], headers=header).json()
        player = pd.json_normalize(player)  
        player = player[['id','fullName','displayName']]
        
        players = pd.concat([players, player], ignore_index=True)      
    
    return players

In [57]:
data = get_active_athletes()

0


In [ ]:
len(data)

In [25]:
data

id             fullName          displayName
0   4429202    Israel Abanikanda    Israel Abanikanda
1   2576336       Ameer Abdullah       Ameer Abdullah
2   3049325      Khalid Abdullah      Khalid Abdullah
3   4360807       Yasir Abdullah       Yasir Abdullah
4   3915373      Micah Abernathy      Micah Abernathy
5     15972         Oday Aboushi         Oday Aboushi
6   4427563        Micah Abraham        Micah Abraham
7   3728305      Johnathan Abram      Johnathan Abram
8   4569234        Keshunn Abram        Keshunn Abram
9   4360643       Delrick Abrams       Delrick Abrams
10  4429193   Kris Abrams-Draine   Kris Abrams-Draine
11  4429160        De'Von Achane        De'Von Achane
12    14152             Sam Acho             Sam Acho
13  4035103         DeMarkus Acy         DeMarkus Acy
14  2576599         Andrew Adams         Andrew Adams
15  4685826             BJ Adams             BJ Adams
16    16800        Davante Adams        Davante Adams
17  3916468           Evan Adams           Evan Adams
18  5084939         Isaiah Adams         Isaiah Adams
19  3115373          Jamal Adams          Jamal Adams
20  2978727         Jerell Adams         Jerell Adams
21  4241053       Jonathan Adams       Jonathan Adams
22  3932420           Josh Adams           Josh Adams
23  3126179        Matthew Adams        Matthew Adams
24  4039164          Myles Adams          Myles Adams
25  3122099           Paul Adams           Paul Adams
26  3059918         Rodney Adams         Rodney Adams
27  4240532           Tony Adams           Tony Adams
28  3886808           Trey Adams           Trey Adams
29  4248067          Tyler Adams          Tyler Adams
30  3895228         Tyrell Adams         Tyrell Adams
31  4261019           Will Adams           Will Adams
32    16039        Jahleel Addae        Jahleel Addae
33  3912028       Nasir Adderley       Nasir Adderley
34  4034729     Tucker Addington     Tucker Addington
35  2971271       Bralon Addison       Bralon Addison
36  4429205       Jordan Addison       Jordan Addison
37    14320        Mario Addison        Mario Addison
38  4427635  Adetomiwa Adebawore  Adetomiwa Adebawore
39  4242547        Paulson Adebo        Paulson Adebo
40  3051869     Quincy Adeboyejo     Quincy Adeboyejo
41  4372754     Jayson Ademilola     Jayson Ademilola
42  4046557       Hakeem Adeniji       Hakeem Adeniji
43  3126081          Ola Adeniyi          Ola Adeniyi
44  3910148         Aaron Adeoye         Aaron Adeoye
45  4426528            Ife Adeyi            Ife Adeyi
46  4383440          Nate Adkins          Nate Adkins
47  4240719         Cal Adomitis         Cal Adomitis
48  4031226      Graham Adomitis      Graham Adomitis
49  2971618       Nelson Agholor       Nelson Agholor

# Play by play API 

In [52]:
import requests

def get_game_plays(game_id: int):

    header = {"User-Agent": "xcs5hg@virginia.edu"}

    event_url = (
        f"https://sports.core.api.espn.com/v2/"
        f"sports/football/leagues/nfl/events/{game_id}"
    )
    event = requests.get(event_url, headers=header).json()

    competition = event["competitions"][0]
    comp_id = competition["id"]

    plays_url = (
        f"https://sports.core.api.espn.com/v2/"
        f"sports/football/leagues/nfl/events/{game_id}/competitions/{comp_id}/plays?limit=500"
    )

    plays = requests.get(plays_url, headers=header).json()
    
    record = []
    
    for item in plays["items"]:
        record.append(item)
        
    keep_cols = [
    "text",
    "period.number",
    "clock.value",
    "start.down",
    "start.distance",
    "start.yardLine",
    "start.yardsToEndzone",
    "end.yardsToEndzone",
    "homeScore",
    "awayScore",
    "statYardage",
    "scoringPlay",
    "scoreValue",
    "type.text",
    "type.abbreviation"]
    
    data = pd.json_normalize(record)
    data = data[keep_cols]  
    
    
    return data

In [53]:
plays = get_game_plays(401671789)
plays


,text,period.number,clock.value,start.down,start.distance,start.yardLine,start.yardsToEndzone,end.yardsToEndzone,homeScore,awayScore,statYardage,scoringPlay,scoreValue,type.text,type.abbreviation
0,GAME,0,900.0,0,0,0,0,65,0,0,0,False,0,Coin Toss,NaN
1,H.Butker kicks 65 yards from KC 35 to end zone...,1,900.0,0,0,35,65,70,0,0,0,False,0,Kickoff,K
2,(Shotgun) D.Henry left end to BLT 32 for 2 yar...,1,900.0,1,10,70,70,68,0,0,2,False,0,Rush,RUSH
3,(Shotgun) L.Jackson pass short right to Z.Flow...,1,859.0,2,8,68,68,73,0,0,-5,False,0,Pass Reception,REC
4,(Shotgun) L.Jackson pass short right to J.Hill...,1,835.0,2,13,73,73,71,0,0,2,False,0,Pass Reception,REC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,"(No Huddle, Shotgun) L.Jackson pass incomplete...",4,18.0,1,10,10,10,10,27,20,0,False,0,Pass Incompletion,NaN
185,Timeout #4 by BLT at 00:10.,4,10.0,2,10,10,10,10,27,20,0,False,0,Timeout,TO
186,(Shotgun) L.Jackson pass incomplete short midd...,4,10.0,2,10,10,10,10,27,20,0,False,0,Pass Incompletion,NaN
187,(Shotgun) L.Jackson pass short middle to I.Lik...,4,5.0,3,10,10,10,10,27,20,0,False,0,Pass Incompletion,NaN


In [54]:
len(plays)

189

## Lou - Testing the NNTRN 

https://gist.github.com/nntrn

In [9]:
import pandas as pd
import requests

In [23]:
import requests
import pandas as pd

# Fetch Athletes
df_athletes = pd.DataFrame(
    requests.get("https://sports.core.api.espn.com/v3/sports/football/nfl/athletes",
                 params={"page": 1, "limit": 20000}).json().get('items', [])
)

# Fetch Events
events = requests.get("https://partners.api.espn.com/v2/sports/football/nfl/events",
                      params={"limit": 1000, "dates": "20230101-20240101"}).json().get('events', [])

df_events = pd.DataFrame([{
    'event_id': e['id'], 'date': e['date'], 'name': e['name'],
    'home_team': (h := next(c for c in e['competitions'][0]['competitors'] if c['homeAway'] == 'home'))['team']['displayName'],
    'home_abbr': h['team']['abbreviation'], 'home_score': h['score']['value'],
    'away_team': (a := next(c for c in e['competitions'][0]['competitors'] if c['homeAway'] == 'away'))['team']['displayName'],
    'away_abbr': a['team']['abbreviation'], 'away_score': a['score']['value'],
    'winner': h['team']['abbreviation'] if h.get('winner') else a['team']['abbreviation'],
    'attendance': e['competitions'][0].get('attendance'),
    'status': e['competitions'][0]['status']['type']['detail']
} for e in events])

# Fetch Fantasy Players
df_fantasy = pd.DataFrame(
    requests.get("https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/2024/players",
                 params={"view": "kona_player_info"},
                 headers={'X-Fantasy-Filter': '{"games":{"limit":2000}}'}).json()
)

print(f"Athletes: {len(df_athletes)}, Events: {len(df_events)}, Fantasy: {len(df_fantasy)}")

Athletes: 19535, Events: 349, Fantasy: 2887


In [25]:
pd.set_option('display.max_columns', None)
df_athletes

,id,uid,guid,lastName,fullName,displayName,shortName,jersey,active,firstName,weight,displayWeight,height,displayHeight,age,dateOfBirth,experience,birthPlace,dateOfDeath,hand,middleName,citizenship,nickname
0,4246273,s:20~l:28~a:4246273,64cf2ccf-8d24-d1be-3ff4-d46499bdd9a6,[35],[35],[35],[35],35,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4246281,s:20~l:28~a:4246281,c6323fce-962f-a112-b136-9a4ed1d4282b,[Downed],[Downed],[Downed],[Downed],00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4246289,s:20~l:28~a:4246289,21d50db4-55e3-a493-d76d-30b5a6786bd5,[Downed],[Downed],[Downed],[Downed],00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4246247,s:20~l:28~a:4246247,5a011f43-549b-7b35-ad26-bb154b973384,[Touchback],[Touchback],[Touchback],[Touchback],00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4246272,s:20~l:28~a:4246272,9efe3029-4e1f-8a43-2035-79e7479f251b,[Touchback],[Touchback],[Touchback],[Touchback],00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19530,16424,s:20~l:28~a:16424,9128af51-f36e-32ba-1fdd-73fcbe2116bc,Zupancic,Mike Zupancic,Mike Zupancic,M. Zupancic,53,False,Mike,240.0,240 lbs,77.0,"6' 5""",35.0,1989-12-14T08:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19531,15462,s:20~l:28~a:15462,0d261997-24bf-5dd1-5e66-93272eed1a44,Zusevics,Markus Zusevics,Markus Zusevics,M. Zusevics,76,False,Markus,300.0,300 lbs,77.0,"6' 5""",36.0,1989-04-25T07:00Z,{'years': 3},NaN,NaN,NaN,NaN,NaN,NaN
19532,11317,s:20~l:28~a:11317,5a6c4114-bbc8-7941-f7f2-ebae32be0e09,Zuttah,Jeremy Zuttah,Jeremy Zuttah,J. Zuttah,64,False,Jeremy,300.0,300 lbs,76.0,"6' 4""",39.0,1986-06-01T07:00Z,{'years': 10},"{'city': 'Edison', 'state': 'NJ'}",NaN,NaN,NaN,NaN,NaN
19533,4294520,s:20~l:28~a:4294520,5114b190-78e6-daca-531b-be29d9cb2e72,Zylstra,Brandon Zylstra,Brandon Zylstra,B. Zylstra,13,False,Brandon,215.0,215 lbs,74.0,"6' 2""",32.0,1993-03-25T08:00Z,{'years': 5},"{'city': 'Spicer', 'state': 'MN', 'country': '...",NaN,NaN,NaN,NaN,NaN


In [71]:
# 1. Get EVENT_IDs
events_response = requests.get(
    "https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events",
    params={"limit": 1000, "dates": "2019"}
).json()

df_events = pd.DataFrame([{
    'event_id': e['$ref'].split('/')[-1].split('?')[0],
    'event_ref': e['$ref']
} for e in events_response['items']])

print(f"Events DataFrame: {len(df_events)} rows")
print(df_events.head())

# 2. Get BET_PROVIDER_IDs
providers_response = requests.get(
    "https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/providers",
    params={"limit": 100}
).json()

# Fetch full details for each provider
provider_data = []
for p in providers_response['items']:
    provider_url = p['$ref']
    try:
        provider_details = requests.get(provider_url).json()
        provider_data.append({
            'provider_id': provider_details['id'],
            'provider_name': provider_details.get('name', 'Unknown'),
            'provider_ref': p['$ref'],
        })
    except Exception as e:
        print(f"Error fetching {provider_url}: {e}")

df_providers = pd.DataFrame(provider_data)

print(f"\nProviders DataFrame: {len(df_providers)} rows")
print(df_providers.head())




Events DataFrame: 333 rows
    event_id                                          event_ref
0  401038952  http://sports.core.api.espn.com/v2/sports/foot...
1  401038953  http://sports.core.api.espn.com/v2/sports/foot...
2  401038955  http://sports.core.api.espn.com/v2/sports/foot...
3  401038954  http://sports.core.api.espn.com/v2/sports/foot...
4  401038957  http://sports.core.api.espn.com/v2/sports/foot...

Providers DataFrame: 71 rows
  provider_id          provider_name  \
0           0          Not Available   
1           1  SportsInteraction.com   
2           2     PinnacleSports.com   
3           3         Sportsbook.com   
4           4              BetUS.COM   

                                        provider_ref  
0  http://sports.core.api.espn.com/v2/sports/foot...  
1  http://sports.core.api.espn.com/v2/sports/foot...  
2  http://sports.core.api.espn.com/v2/sports/foot...  
3  http://sports.core.api.espn.com/v2/sports/foot...  
4  http://sports.core.api.espn.com/v2/sport

In [72]:
df_providers

,provider_id,provider_name,provider_ref
0,0,Not Available,http://sports.core.api.espn.com/v2/sports/foot...
1,1,SportsInteraction.com,http://sports.core.api.espn.com/v2/sports/foot...
2,2,PinnacleSports.com,http://sports.core.api.espn.com/v2/sports/foot...
3,3,Sportsbook.com,http://sports.core.api.espn.com/v2/sports/foot...
4,4,BetUS.COM,http://sports.core.api.espn.com/v2/sports/foot...
...,...,...,...
66,1010,UBet,http://sports.core.api.espn.com/v2/sports/foot...
67,2000,Bet 365,http://sports.core.api.espn.com/v2/sports/foot...
68,2382,Betcris,http://sports.core.api.espn.com/v2/sports/foot...
69,4000,TAB Betting,http://sports.core.api.espn.com/v2/sports/foot...


In [74]:
##  Using the ID '3' for Sportsbook.com  ##
EVENT_ID = df_events.iloc[0]['event_id']
BET_PROVIDER_ID = df_providers.iloc[3]['provider_id']

odds_url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{EVENT_ID}/competitions/{EVENT_ID}/odds/{BET_PROVIDER_ID}/head-to-heads"
main_odds_url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{EVENT_ID}/competitions/{EVENT_ID}/odds"

try:
    main_odds = requests.get(main_odds_url).json()
    print(f"Main odds response: {main_odds}")
    
    if 'items' in main_odds and len(main_odds['items']) > 0:
        # Get available provider IDs from odds
        available_providers = []
        for item in main_odds['items']:
            if 'provider' in item:
                provider_ref = item['provider']['$ref']
                provider_id = provider_ref.split('/')[-1].split('?')[0]
                available_providers.append(provider_id)
        
        print(f"\nAvailable providers with odds: {available_providers}")
        
        # Try first available provider
        if available_providers:
            BET_PROVIDER_ID = available_providers[0]
            odds_url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{EVENT_ID}/competitions/{EVENT_ID}/odds/{BET_PROVIDER_ID}/head-to-heads"
            
            odds_response = requests.get(odds_url).json()
            print(f"\nHead-to-heads response: {odds_response}")
            
            if odds_response['items']:
                df_odds = pd.DataFrame(odds_response['items'])
                print(f"\nOdds DataFrame: {len(df_odds)} rows")
                print(df_odds.head())
            else:
                print("No head-to-head odds available")
                df_odds = pd.DataFrame()
    else:
        print("No odds available for this event")
        df_odds = pd.DataFrame()
        
except Exception as e:
    print(f"\nException: {e}")
    df_odds = pd.DataFrame()

try:
    main_odds = requests.get(main_odds_url).json()
    
    # Convert entire items list to DataFrame using json_normalize to flatten nested structures
    df_odds = pd.json_normalize(main_odds.get('items', []))
    
    print(f"\nOdds DataFrame: {len(df_odds)} rows x {len(df_odds.columns)} columns")
    print(f"\nColumns: {list(df_odds.columns)}")
    print(f"\nFirst few rows:")
    print(df_odds.head())
        
except Exception as e:
    print(f"\nException: {e}")
    df_odds = pd.DataFrame()


Main odds response: {'count': 10, 'pageIndex': 1, 'pageSize': 25, 'pageCount': 1, 'items': [{'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/401038952/competitions/401038952/odds/2000?lang=en&region=us', 'provider': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/providers/2000?lang=en&region=us', 'id': '2000', 'name': 'Bet 365', 'priority': 0}, 'bettingOdds': {'homeTeam': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2018/teams/34?lang=en&region=us'}, 'awayTeam': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2018/teams/11?lang=en&region=us'}, 'teamOdds': {'liveSpreadAway': {'oddId': '1948986771', 'value': '10/11', 'betSlipUrl': 'https://www.bet365.com/dl/sportsbookredirect?affiliate=365_728986&bs=78142645-1948986771~10/11&bet=1'}, 'liveWinningMarginAway': {'oddId': '1949347069', 'value': '17/10', 'betSlipUrl': 'https://www.bet365.com/dl/sportsbookredirect?affil